In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer as stemmer
from tqdm.notebook import tqdm
import rule_book_functs as rbfuncts

In [2]:
incidents_fn = "data/source/20220413_D1_Incidents.csv"
rule_book_fn = "data/rule_book.csv"

In [3]:
rules = pd.read_csv(rule_book_fn)
incidents = pd.read_csv(incidents_fn, dtype=str)

In [4]:
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

incidents = incidents[['incident_id', 'text']]

In [5]:
#sample100 = incidents.sample(n=100)
#sample100.index = range(100)
#sample100.to_csv('sample100.csv', index=False)
sample100 = pd.read_csv('sample100.csv', dtype=str)

In [6]:
#incidents.text[0]

In [7]:
#test = incidents.text.head(1).apply(rbfuncts.lemmatize_text)
#print(list(test))

In [8]:
#test = incidents.text.head(1).apply(rbfuncts.stem_text)
#print(list(test))

In [9]:
#test_tokens = rbfuncts.tokenize(incidents.text[0])

In [10]:
#chk_text = 'employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.'
#pattern = '(\\bemployee (.*) discomfort (.*) eye\\b)'
#rbfuncts.check_presence(pattern, chk_text)

In [11]:
#sents = sent_tokenize(incidents.text[0])
#for sent in sents:
#    if rbfuncts.check_presence(pattern, sent):
#        print(f'{rbfuncts.check_presence(pattern, sent)}: {sent}')

## Load Synonyms & Rules

In [12]:
syn_csv = pd.read_csv('synonyms.csv')

# Load the dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = syn_csv.keywords[r].split(',')
    syn_dict.update({syn_csv.syn[r]:syn_toks})
    
rul_csv = pd.read_csv('rules.csv')

In [13]:
finds_df = rbfuncts.rule_book_scan(sample100, syn_dict, rul_csv)

Checking rule 4 of 7 (slips and trips)


  0%|          | 0/100 [00:00<?, ?it/s]

True: (he(.*)slipped(.*)fell)
True: (scaffolder(.*)slipped(.*)fell)
True: (he(.*)slipped(.*)fell)
True: (scaffolder(.*)slipped(.*)fell)
True: (ip(.*)slipped(.*)fell)
True: (employee(.*)tripped(.*)fell)




In [22]:
strs = 'This is me. He tripped and fell from the ladder.'
#strs = 'during the scaffolding elements transporting for the assembly of scaffolding scaffolder slipped and fell due to the slippery floor'
test = pd.DataFrame({'incident_id':[1052], 'text':[strs]})
rbfuncts.rule_book_scan(test, syn_dict, rul_csv)

Checking rule 4 of 7 (slips and trips)


  0%|          | 0/1 [00:00<?, ?it/s]

,incid_nums,finds_pats,finds_list,incid_cats


In [15]:
print(not False)

True


In [16]:
# How many incidents were classified
len(list(dict.fromkeys(finds_df[finds_df.finds_list == True].incid_nums)))

3

In [17]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='finds')

20172062 ['slips and trips']
during the scaffolding elements transporting for the assembly of scaffolding scaffolder slipped and fell due to the slippery floor .
migrated from legacy cairs incident #32057during the scalffolding elements (tube) transporting for the assembly of scaffolding for fertial shutdown works the scaffolder slipped and fell due to the slippery floor .ip was evacuated with wgs service vehicle to the fertial infirmary he took first aid (alcohol and bandage) due the swollen of his left hand middle finger and then returned to the work station.
migrated from legacy cairs - see detailed description


20201184 ['slips and trips']
team member was exiting water truck backwards while using hand grip, ee slipped and fell, getting a small cut to crown of head.
team member was in the process of exiting the water truck to drive the bus and pick up ee's at the end of the day.
as team member exited the water truck using the hand grip, team members left foot slipped making team me

In [18]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='misses')

20183508 []
ip knocked knee on a telehandler door and on entry to his room.
woke up in the morning with mild swelling and pain on the knee.
first aid case migrated from legacy cairs incident #40135around 15.30 yesterday, ip (nacap)had knocked his left knee on the telehandler door whilst entering the cabin.
ip had continued working normally.around 18.30 yesterday, the ip had knocked the same knee whilst entering his accommodation (block 4 c).
ip had then went for dinner.ip had retired for the evening and woke this morning with mild swelling and pain in the left knee.
ip had then reported the injury and attended the clinic.
ip treated with ibuprofen.
faccorrective / preventativeaction (not working in system)reinforced with crews importance of line of fire and body positioning, reinforced the importance of reporting illnesses and injuries, regardless how minordiscussed longford incident in which a ip suffered a jarred knee.
migrated from legacy cairs - see detailed description


20170361 

20163235 []
a worker was using a mini-excavator when he swung the bolt on the wedge coupler loosened allowing the bucket to disconnect.
no one was in the vicinity and no damage occurred.
migrated from legacy cairs incident #30178a worker operating a 35d mini-excavator had the bucket disconnect as the result of a faulty wedge bolt.
the bolt had become lose as a result of its repetitious use.
no damage was sustained and no harm to personnel existed.
migrated from legacy cairs - see detailed description


20162185 []
cut to finger after pencil grinder jammed ip had been using a pencil grinder in a small work area when the grinder jammed and his finger was caught between the handle shaft and the grinder.
no bony injury.
minor bruising and small laceration to finger.
first aid measures used to treat.
tool was denergised and ip visited medic - first aid measures as treatment.


20211381 []
four (4) weber employees tested positive to covid-19 four (4) weber employees tested positive to covid 

he did not have any time to avoid hitting the deer.
the ee called the hse specialist and reported the incident


20203788 []
wood worker was in monitoring pre embark to pce1 and performed a rt/pcr test on december 19th at the hotel royal macae palace with a positiv wood worker was in monitoring pre embark to pce1 and performed a rt/pcr test on december 19th at the hotel royal macae palace with a positive result.
the ip is asymptomatic.
the worker was kept in the hotel and will be sent home for a quarantine period.


20180162 []
ip's hand was crushed while putting a generator onto the back of a van.
ip's hand was crushed while putting a generator onto the back of a van.ip & his colleague were performing a team lift which is the correct procedure for this type of task but when it came to sitting the generator down they failed to communicate with each other, all the correct ppe was being worn at the time of the accident.
ip attended a&e the following day to get his hand looked at as his h

In [19]:
chk = 'the country gm and regional gm were on their way to their apartment when they were involved in a motor vehicle collision.'
#check_presence('hello(.*)vehicle(.*)collision', chk)
#check_presence('\bregional (.*) were (.*) way\b', chk)
#bool(re.search('\bregional (.*) were (.*) way\b', chk))
bool(re.search('involved(.*)motor(.*)collision', chk))

True

In [20]:
org_string = "This is employee's hand"
pattern = r"'s"

# Replace all occurrences of character 's with an empty string
org_string = re.sub(pattern, '', org_string )
print(org_string)

This is employee hand
